In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


In [2]:
data_dir    = "data/nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " % (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <type 'dict'>, length is 78


data_loader.vocab looks like 
{'\x81': 15, '\x80': 53, '\x83': 49, '\x82': 20, '\x85': 1, '\x84': 5, '\x87': 7, '\x86': 25, '\x89': 43, '\x88': 18, '\x8b': 45, '\n': 22, '\x8d': 34, '\x8c': 33, '\x8f': 8, '\x8e': 17, '\x91': 37, '\x90': 24, '\x93': 14, '\x92': 74, '\x95': 21, '\x94': 23, '\x97': 13, '\x96': 42, '\x99': 55, '\x98': 29, '\x9b': 41, '\x9a': 36, '\x9d': 39, '\x9c': 19, '\x9f': 38, '\x9e': 63, '\xa1': 12, ' ': 6, '\xa3': 10, '\xa2': 28, '\xa5': 3, "'": 51, '>': 77, ')': 48, '(': 47, '-': 67, ',': 30, '.': 27, '\xb1': 11, '0': 75, '\xb3': 64, '\xb2': 32, '\xb5': 52, '\xb4': 2, '\xb7': 16, '\xb6': 46, '\xb9': 9, '\xb8': 35, '\xbb': 57, '\xba': 50, '\xbc': 56, '?': 44, '4': 69, '!': 54, '\x8a': 26, '7': 71, '\xa0': 40, '6': 61, '3': 68, '9': 66, '"': 31, '8': 70, '2': 62, '_': 73, '\xe1': 4, '\xe3': 0, ':': 59, '\x1a': 76, '1': 60, '\xbf': 58, '\xbe': 72, '5': 65} 


type of 'data_loader.chars' i

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

vocab_size = data_loader.vocab_size

# Select RNN Cell
unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)

# Set paths to the graph
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
initial_state = cell.zero_state(batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split(1, seq_length, tf.nn.embedding_lookup(embedding
                    , input_data))
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
# Loop function for seq2seq
def loop(prev, _):
    prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
    prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
    return tf.nn.embedding_lookup(embedding, prev_symbol)
# Output of RNN
outputs, last_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state
                        , cell, loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
# Next word probability
probs = tf.nn.softmax(logits)
# Define LOSS
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight
    vocab_size)
# Define Optimizer
cost = tf.reduce_sum(loss) / batch_size / seq_length
final_state = last_state
lr = tf.Variable(0.0, trainable=False)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

Network Ready


In [ ]:
# Train the model!
num_epochs    = 50
save_every    = 1000
learning_rate = 0.0002
decay_rate    = 0.97

save_dir = '/tmp/tf_logs/char_rnn_hangul'
sess = tf.Session()
sess.run(tf.initialize_all_variables())
summary_writer = tf.train.SummaryWriter(save_dir
                    , graph=sess.graph)
saver = tf.train.Saver(tf.all_variables())
for e in range(num_epochs): # for all epochs

    # Learning rate scheduling
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))

    data_loader.reset_batch_pointer()
    state = sess.run(initial_state)
    for b in range(data_loader.num_batches):
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {input_data: x, targets: y, initial_state: state}
        # Train!
        train_loss, state, _ = sess.run([cost, final_state, optm], feed)
        end = time.time()

        if b % 100 == 0:
            print ("%d/%d (epoch: %d), loss: %.3f, time/batch: %.3f"  
                   % (e * data_loader.num_batches + b
                    , num_epochs * data_loader.num_batches
                    , e, train_loss, end - start))

        if (e * data_loader.num_batches + b) % save_every == 0:
            checkpoint_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path
                       , global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))


0/27500 (epoch: 0), loss: 5.567, time/batch: 2.846
model saved to /tmp/tf_logs/char_rnn_hangul/model.ckpt
